In [1]:
import pandas as pd
import numpy as np
from sklearn import tree,preprocessing

In [2]:
titanic_train = pd.read_csv("train.csv")

titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#if there are any null in age column we can replace them with mean values

new_age = np.where(titanic_train['Age'].isnull(),titanic_train['Age'].median(),titanic_train['Age'])

In [4]:
#identifing the null values

titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [5]:
#LablelEncoder can be used to convert text into binary values

label_encoder = preprocessing.LabelEncoder()

In [6]:
#Converting sex column as binary values

encoded_sex = label_encoder.fit_transform(titanic_train['Sex'])

x = pd.DataFrame(encoded_sex)

y = titanic_train['Survived']

In [7]:
#assigning the decision tree algorithm function to tree_model var

tree_model = tree.DecisionTreeClassifier()

In [8]:
#Training the model

tree_model.fit(x,y)

DecisionTreeClassifier()

In [9]:
#generating decision tree file
 
with open('Dtree.dot','w') as f:
    f = tree.export_graphviz(tree_model,feature_names =['sex'],out_file=f)

In [10]:
# adding Pclass as IDV 

predictors =pd.DataFrame([encoded_sex,titanic_train['Pclass']]).T

In [11]:
tree_model.fit(X=predictors,y = titanic_train['Survived'])

DecisionTreeClassifier()

In [12]:
#generating decision tree file
 
with open('Dtree.dot','w') as f:
    f = tree.export_graphviz(tree_model,feature_names =['sex','Pclass'],out_file=f)

In [13]:
# adding Pclass,Age,Fare as IDV 

predictors =pd.DataFrame([encoded_sex,titanic_train['Pclass'],titanic_train['Age'],titanic_train['Fare']]).T

In [17]:
tree_model.fit(X=predictors,y=titanic_train['Survived'])

DecisionTreeClassifier(max_depth=8)

In [15]:
# since there are 4 IDVs & survived is classified by 0 or 1 we have set the depth as 8 that is 4*2 (Not Mandatory)

tree_model = tree.DecisionTreeClassifier(max_depth=8)

In [18]:
#generating decision tree file

with open('Dtree2.dot','w') as f:
    f = tree.export_graphviz(tree_model,feature_names=['Sex','Pclass','Age','Fare'],out_file=f)

In [43]:
tree_model.score(X=predictors,y=titanic_train['Survived'])

0.8965129358830146

In [19]:
#loading the test set

titanic_test = pd.read_csv("test.csv")

titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [48]:
#identifing the null values

titanic_test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [20]:
#Converting sex column as binary values

encoder_test_sex = label_encoder.fit_transform(titanic_test['Sex'])

In [21]:
test_features = pd.DataFrame([encoder_test_sex,titanic_test['Pclass'],titanic_test['Age'],titanic_test['Fare']]).T

In [22]:
#assigning the prediciton results to a variable

test_pred = tree_model.predict(X=test_features)

In [23]:
pred_op = pd.DataFrame({"passenger_ID" : titanic_test['PassengerId'],"Survived" : test_pred })

pred_op.to_csv('PredictDcTree.csv',index= False)

pred_op.head()

,passenger_ID,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [24]:
# Random forest(ensemble of decision tree) for pruning the decision tree

from sklearn.ensemble import RandomForestClassifier

titanic_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [25]:
# Converting sex & Embarked column as binary values

titanic_train['Sex'] = label_encoder.fit_transform(titanic_train['Sex'])
titanic_train['Embarked'] = label_encoder.fit_transform(titanic_train['Embarked'])

In [26]:
#initializing RF algorithm

rf_model = RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)

In [27]:
features = [ 'Sex','Pclass','SibSp','Embarked','Age','Fare']

In [28]:
rf_model.fit(X=titanic_train[features], y=titanic_train['Survived'])

RandomForestClassifier(max_features=2, n_estimators=1000, oob_score=True)

In [29]:
print("OOB_Accuracy: ",rf_model.oob_score_)

OOB_Accuracy:  0.8098987626546682


In [30]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)

Sex 0.2688090939586464
Pclass 0.0884193750033036
SibSp 0.05264093842789755
Embarked 0.03372371802286874
Age 0.26856168538902736
Fare 0.2878451891982565


# Project 1 : Build Decision Tree(DV-"Survived",IDV-"Age,Gender and Fare") and Prediction

In [31]:
#Creating a DF with IDV variables 

predictors =pd.DataFrame([encoded_sex,titanic_train['Age'],titanic_train['Fare']]).T

predictors.head()

,0,1,2
0,1.0,22.0,7.2500
1,0.0,38.0,71.2833
2,0.0,26.0,7.9250
3,0.0,35.0,53.1000
4,1.0,35.0,8.0500


In [36]:
#initializing DecisionTreeClassifier algorithm 

dc_tree = tree.DecisionTreeClassifier(max_depth=8)

In [37]:
dc_tree.fit(X=predictors,y=titanic_train['Survived'])

DecisionTreeClassifier(max_depth=8)

In [38]:
#generating decision tree file

with open('Dtree3.dot','w') as f:
    f = tree.export_graphviz(dc_tree,feature_names=['Sex','Age','Fare'],out_file=f)

In [39]:
dc_tree.score(X=predictors,y=titanic_train['Survived'])

0.8706411698537683

In [40]:
#Testing the model

test_feature = pd.DataFrame([encoder_test_sex,titanic_test['Age'],titanic_test['Fare']]).T

test_feature.head()

,0,1,2
0,1.0,34.5,7.8292
1,0.0,47.0,7.0000
2,1.0,62.0,9.6875
3,1.0,27.0,8.6625
4,0.0,22.0,12.2875


In [42]:
#predicting survival rate

pred_vl = dc_tree.predict(X=test_feature)

In [43]:
pred_opp = pd.DataFrame({"passenger_ID" : titanic_test['PassengerId'],"Survived" : pred_vl })

pred_opp.head()

,passenger_ID,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
